In [157]:
import numpy as np
from sklearn import tree,metrics,svm,ensemble,neighbors
import pandas as pd
from sklearn.model_selection import train_test_split,KFold
from sklearn import linear_model
import xgboost as xgb
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from scipy import optimize

In [183]:
mixture = ['氮化钒铁FeV55N11-A','低铝硅铁','钒氮合金(进口)','钒铁(FeV50-A)','钒铁(FeV50-B)','硅铝钙','硅铝合金FeAl30Si25','硅铝锰合金球',
           '硅锰面（硅锰渣）','硅铁(合格块)','硅铁FeSi75-B','石油焦增碳剂','锰硅合金FeMn64Si27(合格块)','锰硅合金FeMn68Si18(合格块)',
           '碳化硅(55%)','硅钙碳脱氧剂']
columns = ['转炉终点温度', '转炉终点C', '转炉终点Mn', '转炉终点S', '转炉终点P',
       '转炉终点Si', '钢水净重', '连铸正样C', '连铸正样Mn', '连铸正样S', '连铸正样P', '连铸正样Si', '氮化钒铁FeV55N11-A', '低铝硅铁',
       '钒氮合金(进口)', '钒铁(FeV50-A)', '钒铁(FeV50-B)', '钒铁(FeV50-B).1', '硅铝钙',
       '硅铝合金FeAl30Si25', '硅铝锰合金球', '硅锰面（硅锰渣）', '硅铁(合格块)', '硅铁FeSi75-B',
       '石油焦增碳剂', '锰硅合金FeMn64Si27(合格块)', '锰硅合金FeMn68Si18(合格块)', '碳化硅(55%)',
       '硅钙碳脱氧剂']
x_col = [ '转炉终点C', '转炉终点Mn', '转炉终点S', '转炉终点P',
       '转炉终点Si','转炉终点温度', '钢水净重', '氮化钒铁FeV55N11-A', '低铝硅铁',
       '钒氮合金(进口)', '钒铁(FeV50-A)', '钒铁(FeV50-B)', '硅铝钙',
       '硅铝合金FeAl30Si25', '硅铝锰合金球', '硅锰面（硅锰渣）', '硅铁(合格块)', '硅铁FeSi75-B',
       '石油焦增碳剂', '锰硅合金FeMn64Si27(合格块)', '锰硅合金FeMn68Si18(合格块)', '碳化硅(55%)',
       '硅钙碳脱氧剂']
price = [350000,6500,350000,205000,205000,11800,1000,8500,7600,6000,6000,4600,8150,8150,6100,4000]
len(price)

16

In [281]:
data.std()

转炉终点温度                  412.509148
转炉终点C                     0.000212
转炉终点Mn                    0.000142
转炉终点S                     0.000057
转炉终点P                     0.000040
转炉终点Si                    0.000773
钢水净重                   3450.728682
连铸正样C                     0.000068
连铸正样Mn                    0.000330
连铸正样S                     0.000038
连铸正样P                     0.000055
连铸正样Si                    0.000205
氮化钒铁FeV55N11-A           10.857237
低铝硅铁                      0.000000
钒氮合金(进口)                  7.862632
钒铁(FeV50-A)               3.076156
钒铁(FeV50-B)              12.879600
硅铝钙                      20.937015
硅铝合金FeAl30Si25           12.292978
硅铝锰合金球                    0.000000
硅锰面（硅锰渣）                103.423766
硅铁(合格块)                   0.000000
硅铁FeSi75-B                1.375760
石油焦增碳剂                   17.119141
锰硅合金FeMn64Si27(合格块)     251.146294
锰硅合金FeMn68Si18(合格块)     305.531566
碳化硅(55%)                 24.048310
硅钙碳脱氧剂                    7.214721
dtype: float64

In [284]:
data = pd.read_excel('./data1.xlsx')
data.drop(data[data['连铸正样C'].isnull()].index,inplace=True)
data=data[data['钢号']=='HRB400B    '][columns]
# 离群值处理
outs=['转炉终点温度', '转炉终点C', '转炉终点Mn', '转炉终点S', '转炉终点P',
       '转炉终点Si', '钢水净重', '连铸正样C', '连铸正样Mn', '连铸正样S', '连铸正样P', '连铸正样Si']
for out in outs:
    data.drop(data[(data[out]>(data[out].mean()+3*data[out].std()))|(data[out]<(data[out].mean()-3*data[out].std()))].index,inplace=True)

data.fillna(data.mean(),inplace=True)
data2 = pd.read_excel('data2.xlsx')
data2.head(2)
data['钒铁(FeV50-B)'] = data['钒铁(FeV50-B)']+data['钒铁(FeV50-B).1']
data.drop('钒铁(FeV50-B).1',axis=1,inplace=True)
# X = data[x_col].apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)+1e-10)).values
X = data[x_col].values

len(data)#.isna().sum()

571

In [285]:
# data[x_col].apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)+1e-10))

### 以下分别计算五种元素的预测模型

In [295]:
Y = data['连铸正样Mn'].values
clf = linear_model.BayesianRidge()
ki = 8
kf = KFold(n_splits=ki)
r2,mae,mse=0,0,0
for train_index,test_index in kf.split(X):
    clf_Mn = clf.fit(X[train_index],Y[train_index])
    pre_y = clf_Mn.predict(X[test_index])
    r2+=metrics.r2_score(Y[test_index],pre_y)
    mae+=metrics.mean_absolute_error(Y[test_index],pre_y)
    mse+=metrics.mean_squared_error(Y[test_index],pre_y)
## r2决定系数，绝对误差，绝对均方误差，绝对误差百分比
r2/ki,mae/ki,mse/ki,np.mean(np.abs((Y[test_index]-pre_y)/Y[test_index]))

(-0.06076393145840901,
 0.00023929442058404156,
 8.973495987630443e-08,
 0.02089343375638583)

In [287]:
Y = data['连铸正样C'].values
clf = linear_model.LinearRegression()
ki = 8
kf = KFold(n_splits=ki)
r2,mae,mse=0,0,0
for train_index,test_index in kf.split(X):
    clf_C = clf.fit(X[train_index],Y[train_index])
    pre_y = clf_C.predict(X[test_index])
    r2+=metrics.r2_score(Y[test_index],pre_y)
    mae+=metrics.mean_absolute_error(Y[test_index],pre_y)
    mse+=metrics.mean_squared_error(Y[test_index],pre_y)
r2/ki,mae/ki,mse/ki,np.mean(np.abs((Y[test_index]-pre_y)/Y[test_index]))

(-0.2100922490871883,
 5.9663771609769454e-05,
 4.984185029925866e-09,
 0.02457665056753315)

In [288]:
Y = data['连铸正样S'].values
clf = linear_model.LinearRegression()
ki = 8
kf = KFold(n_splits=ki)
r2,mae,mse=0,0,0
for train_index,test_index in kf.split(X):
    clf_S = clf.fit(X[train_index],Y[train_index])
    pre_y = clf_S.predict(X[test_index])
    r2+=metrics.r2_score(Y[test_index],pre_y)
    mae+=metrics.mean_absolute_error(Y[test_index],pre_y)
    mse+=metrics.mean_squared_error(Y[test_index],pre_y)
r2/ki,mae/ki,mse/ki,np.mean(np.abs((Y[test_index]-pre_y)/Y[test_index]))

(0.22971568936512532,
 2.4039164483591584e-05,
 9.160905569484165e-10,
 0.07204753970427419)

In [289]:
Y = data['连铸正样P'].values
clf = linear_model.BayesianRidge()
ki = 8
kf = KFold(n_splits=ki)
r2,mae,mse=0,0,0
for train_index,test_index in kf.split(X):
    clf_P = clf.fit(X[train_index],Y[train_index])
    pre_y = clf_P.predict(X[test_index])
    r2+=metrics.r2_score(Y[test_index],pre_y)
    mae+=metrics.mean_absolute_error(Y[test_index],pre_y)
    mse+=metrics.mean_squared_error(Y[test_index],pre_y)
r2/ki,mae/ki,mse/ki,np.mean(np.abs((Y[test_index]-pre_y)/Y[test_index]))

(-0.07093124200899147,
 4.448433199415444e-05,
 3.0275510127866136e-09,
 0.22312579203509067)

In [290]:
Y = data['连铸正样Si'].values
clf = linear_model.LinearRegression()
ki = 8
kf = KFold(n_splits=ki)
r2,mae,mse=0,0,0
for train_index,test_index in kf.split(X):
    clf_Si = clf.fit(X[train_index],Y[train_index])
    pre_y = clf_Si.predict(X[test_index])
    r2+=metrics.r2_score(Y[test_index],pre_y)
    mae+=metrics.mean_absolute_error(Y[test_index],pre_y)
    mse+=metrics.mean_squared_error(Y[test_index],pre_y)
r2/ki,mae/ki,mse/ki,np.mean(np.abs((Y[test_index]-pre_y)/Y[test_index]))

(0.11498135556511822,
 0.0001410005287122211,
 3.100955766429374e-08,
 0.03522386820506117)

## 使用常量替代不是配料的变量，计算一吨合金在下列条件下的最低配料费用

In [291]:
init_ = [data['转炉终点C'].mean(),data['转炉终点Mn'].mean(),data['转炉终点S'].mean(),data['转炉终点P'].mean(),data['转炉终点Si'].mean(),data['转炉终点温度'].mean(),1000]

## 建立线性约束不等式以及目标函数,并优化求解

In [293]:
c = data2['价格(元/吨)'].values/1000 #配料价格作为目标函数系数/kg
a = np.array([
#     list(clf_C.coef_[7:]),list(-clf_C.coef_[7:]),list(clf_Mn.coef_[7:]),list(-clf_Mn.coef_[7:]),
    list(clf_S.coef_[7:]),list(clf_P.coef_[7:])
#     ,list(clf_Si.coef_[7:]),list(-clf_Si.coef_[7:])
])
b = np.array([
#     0.0025-sum(clf_C.coef_[:7]*init_),-0.0019+sum(clf_C.coef_[:7]*init_),
#     0.016-sum(clf_Mn.coef_[:7]*init_),-0.013+sum(clf_Mn.coef_[:7]*init_),
    0.00045-sum(clf_S.coef_[:7]*init_),0.00045-sum(clf_P.coef_[:7]*init_),
#     0.0065-sum(clf_Si.coef_[:7]*init_),-0.005+sum(clf_Si.coef_[:7]*init_)
])

a_ = np.array([
    list(clf_C.coef_[7:]),list(clf_Mn.coef_[7:]),
#     list(clf_S.coef_[7:]),list(clf_P.coef_[7:])
    list(clf_Si.coef_[7:])
])
b_ = np.array([
    0.0023-sum(clf_C.coef_[:7]*init_),
    0.0145-sum(clf_Mn.coef_[:7]*init_),
#     0.00045-sum(clf_S.coef_[:7]*init_),
#     0.00045-sum(clf_P.coef_[:7]*init_),
    0.0055-sum(clf_Si.coef_[:7]*init_)
])
bds = [(0,None)]*len(list(clf_C.coef_[7:]))
optimize.linprog(c,A_ub=a,b_ub=b,bounds=bds,A_eq=a_,b_eq=b_)

     fun: 661570.8133005378
 message: 'Optimization terminated successfully.'
     nit: 13
   slack: array([0.00037447, 0.        ])
  status: 0
 success: True
       x: array([    0.        ,     0.        ,     0.        ,     0.        ,
           0.        ,     0.        ,   980.69353171,     0.        ,
           0.        ,     0.        ,   177.99445769,     0.        ,
       66524.75305823, 14398.21050284,     0.        ,     0.        ])